## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [2]:
df = pd.read_csv('KK_mortgage_map - Sheet1.csv')

## Google Maps API Geolocater Setup

In [3]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [4]:
df['geo_address'] = df['full_address'] + ' Miami-Dade, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [5]:
df.columns

Index(['full_address', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [6]:
df

,full_address,geo_address,loc,point,lat,lon,altitude
0,2308 Northwest Fifth Avenue,"2308 Northwest Fifth Avenue Miami-Dade, FL","(2308 NW 5 Ave, Miami, FL 33127, USA, (25.7992...","(25.7992793, -80.20343729999999, 0.0)",25.799279,-80.203437,0.0
1,2300 Northwest Fifth Avenue,"2300 Northwest Fifth Avenue Miami-Dade, FL","(2300 NW 5 Ave, Miami, FL 33127, USA, (25.7992...","(25.7992418, -80.2033776, 0.0)",25.799242,-80.203378,0.0
2,2380 Northwest Fifth Avenue508 Northwest 23rd ...,2380 Northwest Fifth Avenue508 Northwest 23rd ...,"(2380 NW 5th St, Miami, FL 33125, USA, (25.777...","(25.777599, -80.2352329, 0.0)",25.777599,-80.235233,0.0
3,514 Northwest 23rd Street,"514 Northwest 23rd Street Miami-Dade, FL","(514 NW 23rd St, Miami, FL 33127, USA, (25.798...","(25.798824, -80.203464, 0.0)",25.798824,-80.203464,0.0
4,518 Northwest 23rd Street2320 Northwest Fifth ...,518 Northwest 23rd Street2320 Northwest Fifth ...,"(518 NW 23rd St #2320, Miami, FL 33127, USA, (...","(25.798818, -80.20355599999999, 0.0)",25.798818,-80.203556,0.0
5,2324 Northwest Fifth Avenue,"2324 Northwest Fifth Avenue Miami-Dade, FL","(2324 NW 5 Ave, Miami, FL 33127, USA, (25.7995...","(25.799511, -80.2034809, 0.0)",25.799511,-80.203481,0.0
6,2330 Northwest Fifth Avenue490 Northwest 23rd ...,2330 Northwest Fifth Avenue490 Northwest 23rd ...,"(2330 NW 5th St, Miami, FL 33125, USA, (25.777...","(25.777815, -80.2352155, 0.0)",25.777815,-80.235215,0.0
7,435 Northwest 22nd Lane,"435 Northwest 22nd Lane Miami-Dade, FL","(435 NW 22nd Ln, Miami, FL 33127, USA, (25.798...","(25.7989949, -80.2021675, 0.0)",25.798995,-80.202168,0.0
8,592 Northwest 22nd Street,"592 Northwest 22nd Street Miami-Dade, FL","(592 NW 22nd St, Miami, FL 33127, USA, (25.797...","(25.797842, -80.204814, 0.0)",25.797842,-80.204814,0.0
9,598 Northwest 22nd Street,"598 Northwest 22nd Street Miami-Dade, FL","(598 NW 22nd St, Miami, FL 33127, USA, (25.797...","(25.79784, -80.204927, 0.0)",25.797840,-80.204927,0.0


## Correction section

In [78]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [79]:
df.columns

Index(['full_address', 'developers', 'project_name', 'sq_ft', 'description',
       'story_link', 'addtl_links', 'other_notes', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [7]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [81]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [82]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/10/11/south-beach-office-project-backed-by-google-ex-ceo-eric-schmidt-launches-leasing/" target="_blank" rel="noopener noreferrer">The Fifth Miami</a> Beach is a partnership among Sumaida + Khurana and Bizzi & Partners, both based in New York, as well as ex-Google CEO Eric Schmidt who together with his wife has an 88 percent stake in the entity that owns the developments sites. The two New York-based firms have a 12 percent stake. It\'s a five-story office building with 3,125 square feet of ground-floor retail'

In [83]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Office Projects in Miami Beach")

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [84]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [85]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [86]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miami_beach_office_projects
